# Metrics with multiple references
The following are multiple-reference based metrics.

*   SARI
*   BertScore
*   SARI



In [9]:
from google.colab import files
!pip install textstat
!pip install --upgrade transformers
!pip install simpletransformers
!pip install transformers
!pip install datasets
!pip install evaluate

In [10]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
#from simpletransformers.classification import ClassificationModel, ClassificationArgs
import numpy as np
import pandas as pd
import os
from google.colab import drive
drive.mount('/content/drive')
!ls '/content/drive/MyDrive/TEC/Proyecto/Investigacion'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 Alpaca						  'Instrucciones de_Requerimientos_de_Sistema.pdf'
'Analisis cualitativo de seleccion.gdoc'	   Instrucciones_Plan_de_Proyecto.pdf
'Aplicacion Metricas completas.ipynb'		   Instrucciones_Requerimientos_de_Software.pdf
'BRENES BERTA - Resultado Ficha Proyecto.pdf'	   Investigacion.ipynb
 Comparacion_simplificación_sample.xlsx		   llama
 Dataset5000					   Llama
 dataset_replicas2.csv				   manually_utf8.csv
 dataset_replicas3.csv				   Mavin_A_Rolls_Royce_EARS_RE09_Paperaccepted.pdf
 dataset_replicas4.csv				   Metrics_sari_bleu.ipynb
 dataset_replicas5.csv				   Pegasus.ipynb
'Documento de Diseño-Bertha Brenes.gdoc'	  'Plan de proyecto.gdoc'
'Documento de requerimiento.gdoc'		   Poster.gdoc
'Documento para el profe.gdoc'			  'Probando el dataset.ipynb'
 easse						  'Recoleccion de datos.gdoc'
 FastChat.ipynb					   salidaPegasus2.csv
'Ficha de 

In [11]:
!git clone https://github.com/feralvam/easse.git



fatal: destination path 'easse' already exists and is not an empty directory.


In [12]:
#%cd easse
%ls


demo/   example.sh*  MANIFEST.in  requirements.txt  setup.py
easse/  LICENSE      README.md    setup.cfg         tests/


In [13]:
!ls
!pip install -e .

demo   example.sh  MANIFEST.in	requirements.txt  setup.py
easse  LICENSE	   README.md	setup.cfg	  tests
Obtaining file:///content/easse
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/facebookresearch/text-simplification-evaluation.git (to revision main) to /tmp/pip-install-7q6em7_6/tseval_8ed1cb882cff415eba3ad667a65f3f41
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/text-simplification-evaluation.git /tmp/pip-install-7q6em7_6/tseval_8ed1cb882cff415eba3ad667a65f3f41
  Resolved https://github.com/facebookresearch/text-simplification-evaluation.git to commit f335e2e27026321c7c3d1dd63857416c7e7397b2
  Preparing metadata (setup.py) ... done
  Using cached sacrebleu-2.3.1-py3-none-any.whl (118 kB)
  Using cached sacremoses-0.0.53.tar.gz (880 kB)
  Preparing metadata (setup.py) ... done
  Using cached stanfordnlp-0.2.0-py3-none-any.whl (158 kB)
  Using cached yattag-1.15.1.tar.gz (28 kB)
  Preparing metadata (setup.py) ... do

In [14]:
!sudo apt-get install texlive-latex-recommended

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  dvisvgm fonts-droid-fallback fonts-lmodern fonts-noto-mono fonts-urw-base35
  libfontenc1 libgs9 libgs9-common libidn12 libijs-0.35 libjbig2dec0
  libkpathsea6 libptexenc1 libsynctex2 libteckit0 libtexlua53 libtexluajit2
  libwoff1 libzzip-0-13 lmodern poppler-data t1utils tex-common texlive-base
  texlive-binaries texlive-latex-base xfonts-encodings xfonts-utils
Suggested packages:
  fonts-noto fonts-freefont-otf | fonts-freefont-ttf fonts-texgyre
  poppler-utils ghostscript fonts-japanese-mincho | fonts-ipafont-mincho
  fonts-japanese-gothic | fonts-ipafont-gothic fonts-arphic-ukai
  fonts-arphic-uming fonts-nanum debhelper gv | postscript-viewer perl-tk xpdf
  | pdf-viewer xzdec texlive-latex-base-doc texlive-latex-recommended-doc
  texlive-luatex texlive-pstricks
The following NEW packages will be installed:
  dvisvgm fonts-droid-fa

In [15]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [19]:
import pandas as pd
from easse.sari import corpus_sari
from easse.bleu import corpus_bleu
from easse.bertscore import corpus_bertscore
from pathlib import Path

def calculate_metrics_multiple_references(path_file_simplifications, path_file_references,columnName= "simple",complexName="complex", num_replicas = 4):
  data_frame_simplifications =  pd.read_csv(path_file_simplifications, encoding="utf_8")
  data_frame_references =  pd.read_csv(path_file_references)
  sari_sentences = []
  bleu_sentences = []
  berts_sentences = []
  #go through each sentence in the simplifications file and look for its references
  for chunk_data_frame in pd.read_csv(path_file_references, chunksize=1):
    #id current sentence
    id_sentence =  str(chunk_data_frame["idFinal"].values[0])
    #get simple texts in spanish
    #this is an nd array
    #verify if the validation sentences is in the simplified dataset
    num_hits = len(data_frame_simplifications[data_frame_simplifications["id_final"] == id_sentence])
    #print("num_hits ", num_hits)
    if( num_hits > 0):
      simple_text_array = str(data_frame_simplifications[data_frame_simplifications["id_final"] == id_sentence][columnName].values[0])
      complex_text_array = str(data_frame_simplifications[data_frame_simplifications["id_final"] == id_sentence][complexName].values[0])

      #get the reference sentences to calculate the metrics
      list_references = []
      for i in range(0, num_replicas):
        simple_column = "simple" + str(i + 1)
        list_references += [[data_frame_references[data_frame_references["idFinal"] == id_sentence][simple_column].iloc[0]]]
      #calculate the metrics
      orig_sents = [complex_text_array]
      sys_sents = [simple_text_array]
      sari_metric = corpus_sari(orig_sents=orig_sents,
              sys_sents=sys_sents,
              refs_sents=list_references)

      bleu_metric = corpus_bleu(sys_sents=sys_sents, refs_sents=list_references)

      #bert_score = corpus_bertscore(sys_sents=sys_sents, refs_sents=list_references)
      #save all metrics
      sari_sentences += [sari_metric]
      bleu_sentences += [bleu_metric]
      #berts_sentences += [bert_score]

  return np.array(sari_sentences), np.array(bleu_sentences)#, np.array(berts_sentences)



def generate_multi_ref_stats(path_file_references, path_file_simplifications, columnName, complexName):
  name_file_latex = Path(path_file_references).stem + "_" + Path(path_file_simplifications).stem + "_stats.tex"

  #header of latex table
  table_values = np.zeros((4, 8))
  col_labels = ["Num. refs.", "Num. obs.", "mu sari", "std sari", "mu bleu", "std bleu", "mu bert", "std bert"]
  dict_keys_sari = ["sari_2", "sari_3", "sari_4", "sari_5", "sari_6"]
  dict_keys_bleu = ["bleu_2", "bleu_3", "bleu_4", "bleu_5", "bleu_6"]
  dict_sari = {}
  dict_bleu = {}
  for i in range(2, 6):
    print("Num references ", i)
    #calculate metrics
    path_file_references_curr =  path_file_references + str(i) + ".csv"
    sari_sentences, bleu_sentences = calculate_metrics_multiple_references(path_file_simplifications, path_file_references_curr, columnName,complexName ,num_replicas = 2)
    print("num sentences: ", len(sari_sentences))
    table_values[i - 2, 0] = i
    table_values[i - 2, 1] = len(sari_sentences)
    table_values[i - 2, 2] = np.mean(sari_sentences)
    table_values[i - 2, 3] = np.std(sari_sentences)
    table_values[i - 2, 4] = np.mean(bleu_sentences)
    table_values[i - 2, 5] = np.std(bleu_sentences)
    #table_values[i - 2, 6] = np.mean(bert_sentences)
    #table_values[i - 2, 7] = np.std(bert_sentences)
    dict_sari[dict_keys_sari[i -2]] = sari_sentences
    dict_bleu[dict_keys_bleu[i -2]] = bleu_sentences

  #data_frame_all_bleu = pd.
  #put data on dataframe
  table_values = np.round(table_values, decimals = 2)
  data_frame = pd.DataFrame(table_values, columns = col_labels)
  data_frame.reset_index(drop = True)
  data_frame.round(decimals = 2)
  print("Data frame \n ", data_frame)
  #store latex file
  with open(name_file_latex,'w') as tf:
    tf.write(data_frame.to_latex())

  return table_values, col_labels


## Tests with Pegasus

In [23]:
path_file_references = "/content/drive/MyDrive/TEC/Proyecto/Investigacion/dataset_replicas"
path_file_simplifications = "/content/drive/MyDrive/TEC/Proyecto/Investigacion/Dataset5000/salidaPegasus.csv"
#path_file_simplifications = "/content/drive/MyDrive/TEC/Proyecto/Investigacion/Dataset5000/salidaPegasus_Finalrep.csv"
#dict_sari_pegasus, dict_bleu_pegasus = generate_multi_ref_stats(path_file_references, path_file_simplifications, "simple","Segmento" )
dict_sari_pegasus, dict_bleu_pegasus = generate_multi_ref_stats(path_file_references, path_file_simplifications, "simplePegasus","complex" )
df = pd.DataFrame(dict_sari_pegasus, columns=dict_bleu_pegasus)
df

Num references  2
num sentences:  149
Num references  3
num sentences:  134
Num references  4
num sentences:  119
Num references  5
num sentences:  94
Data frame 
     Num. refs.  Num. obs.  mu sari  std sari  mu bleu  std bleu  mu bert  \
0         2.0      149.0    30.45      8.59    18.07     16.00      0.0   
1         3.0      134.0    30.57      8.37    16.73     14.68      0.0   
2         4.0      119.0    31.34      8.22    16.76     14.66      0.0   
3         5.0       94.0    31.75      8.18    16.56     14.35      0.0   

   std bert  
0       0.0  
1       0.0  
2       0.0  
3       0.0  


<ipython-input-19-24ff55f53c70>:86: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  tf.write(data_frame.to_latex())


,Num. refs.,Num. obs.,mu sari,std sari,mu bleu,std bleu,mu bert,std bert
0,2.0,149.0,30.45,8.59,18.07,16.00,0.0,0.0
1,3.0,134.0,30.57,8.37,16.73,14.68,0.0,0.0
2,4.0,119.0,31.34,8.22,16.76,14.66,0.0,0.0
3,5.0,94.0,31.75,8.18,16.56,14.35,0.0,0.0


## Tests with Manually simplified dataset

In [24]:
path_file_references = "/content/drive/MyDrive/TEC/Proyecto/Investigacion/dataset_replicas"
path_file_simplifications = "/content/drive/MyDrive/TEC/Proyecto/Investigacion/manually_utf8.csv"
dict_sari_manual, dict_bleu_manual = generate_multi_ref_stats(path_file_references, path_file_simplifications,"simple", "complex")
df = pd.DataFrame(dict_sari_manual, columns=dict_bleu_manual)
df

Num references  2
num sentences:  149
Num references  3
num sentences:  134
Num references  4
num sentences:  119
Num references  5
num sentences:  94
Data frame 
     Num. refs.  Num. obs.  mu sari  std sari  mu bleu  std bleu  mu bert  \
0         2.0      149.0    48.72     12.66    49.01     23.17      0.0   
1         3.0      134.0    48.29     11.97    48.53     22.59      0.0   
2         4.0      119.0    48.16     11.92    47.51     22.33      0.0   
3         5.0       94.0    48.87     10.72    47.93     20.92      0.0   

   std bert  
0       0.0  
1       0.0  
2       0.0  
3       0.0  


<ipython-input-19-24ff55f53c70>:86: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  tf.write(data_frame.to_latex())


,Num. refs.,Num. obs.,mu sari,std sari,mu bleu,std bleu,mu bert,std bert
0,2.0,149.0,48.72,12.66,49.01,23.17,0.0,0.0
1,3.0,134.0,48.29,11.97,48.53,22.59,0.0,0.0
2,4.0,119.0,48.16,11.92,47.51,22.33,0.0,0.0
3,5.0,94.0,48.87,10.72,47.93,20.92,0.0,0.0


## Tests with Llama

In [26]:
path_file_references = "/content/drive/MyDrive/TEC/Proyecto/Investigacion/dataset_replicas"
path_file_simplifications = "/content/drive/MyDrive/TEC/Proyecto/Investigacion/Dataset5000/salidaLlama.csv"
dict_sari_mt5, dict_bleu_mt5 = generate_multi_ref_stats(path_file_references, path_file_simplifications, "simpleLlama", "complex")
df = pd.DataFrame(dict_sari_mt5, columns=dict_bleu_mt5)
df

Num references  2
num sentences:  149
Num references  3
num sentences:  134
Num references  4
num sentences:  119
Num references  5
num sentences:  94
Data frame 
     Num. refs.  Num. obs.  mu sari  std sari  mu bleu  std bleu  mu bert  \
0         2.0      149.0    21.50      4.28    59.32     16.09      0.0   
1         3.0      134.0    21.42      4.32    59.60     15.46      0.0   
2         4.0      119.0    21.16      4.36    58.79     15.61      0.0   
3         5.0       94.0    21.31      4.09    59.25     14.37      0.0   

   std bert  
0       0.0  
1       0.0  
2       0.0  
3       0.0  


<ipython-input-19-24ff55f53c70>:86: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  tf.write(data_frame.to_latex())


,Num. refs.,Num. obs.,mu sari,std sari,mu bleu,std bleu,mu bert,std bert
0,2.0,149.0,21.50,4.28,59.32,16.09,0.0,0.0
1,3.0,134.0,21.42,4.32,59.60,15.46,0.0,0.0
2,4.0,119.0,21.16,4.36,58.79,15.61,0.0,0.0
3,5.0,94.0,21.31,4.09,59.25,14.37,0.0,0.0


## Tests with FastChat

In [27]:
path_file_references = "/content/drive/MyDrive/TEC/Proyecto/Investigacion/dataset_replicas"
path_file_simplifications = "/content/drive/MyDrive/TEC/Proyecto/Investigacion/Dataset5000/salidaFastChat.csv"
dict_sari_tuner, dict_bleu_tuner = generate_multi_ref_stats(path_file_references, path_file_simplifications, "simpleFastChat", "complex")
df = pd.DataFrame(dict_sari_tuner, columns=dict_bleu_tuner)
df

Num references  2
num sentences:  3
Num references  3
num sentences:  3
Num references  4
num sentences:  2
Num references  5
num sentences:  1
Data frame 
     Num. refs.  Num. obs.  mu sari  std sari  mu bleu  std bleu  mu bert  \
0         2.0        3.0    24.73      0.80    71.47      8.73      0.0   
1         3.0        3.0    24.73      0.80    71.47      8.73      0.0   
2         4.0        2.0    24.79      0.98    77.63      0.55      0.0   
3         5.0        1.0    25.76      0.00    78.18      0.00      0.0   

   std bert  
0       0.0  
1       0.0  
2       0.0  
3       0.0  


<ipython-input-19-24ff55f53c70>:86: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  tf.write(data_frame.to_latex())


,Num. refs.,Num. obs.,mu sari,std sari,mu bleu,std bleu,mu bert,std bert
0,2.0,3.0,24.73,0.80,71.47,8.73,0.0,0.0
1,3.0,3.0,24.73,0.80,71.47,8.73,0.0,0.0
2,4.0,2.0,24.79,0.98,77.63,0.55,0.0,0.0
3,5.0,1.0,25.76,0.00,78.18,0.00,0.0,0.0


## Tests with Alpaca

In [28]:
path_file_references = "/content/drive/MyDrive/TEC/Proyecto/Investigacion/dataset_replicas"
path_file_simplifications = "/content/drive/MyDrive/TEC/Proyecto/Investigacion/Dataset5000/salidaAlpaca.csv"
dict_sari_tuner, dict_bleu_tuner = generate_multi_ref_stats(path_file_references, path_file_simplifications, "simpleAlpaca", "complex")
df = pd.DataFrame(dict_sari_tuner, columns=dict_bleu_tuner)
df

Num references  2
num sentences:  149
Num references  3
num sentences:  134
Num references  4
num sentences:  119
Num references  5
num sentences:  94
Data frame 
     Num. refs.  Num. obs.  mu sari  std sari  mu bleu  std bleu  mu bert  \
0         2.0      149.0    21.40      4.07    55.54     14.91      0.0   
1         3.0      134.0    21.33      4.08    56.18     14.52      0.0   
2         4.0      119.0    21.06      4.09    55.54     14.73      0.0   
3         5.0       94.0    21.19      3.72    56.01     13.68      0.0   

   std bert  
0       0.0  
1       0.0  
2       0.0  
3       0.0  


<ipython-input-19-24ff55f53c70>:86: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  tf.write(data_frame.to_latex())


,Num. refs.,Num. obs.,mu sari,std sari,mu bleu,std bleu,mu bert,std bert
0,2.0,149.0,21.40,4.07,55.54,14.91,0.0,0.0
1,3.0,134.0,21.33,4.08,56.18,14.52,0.0,0.0
2,4.0,119.0,21.06,4.09,55.54,14.73,0.0,0.0
3,5.0,94.0,21.19,3.72,56.01,13.68,0.0,0.0


# Statistical analysis
We first check whether equal variances and normality is met

In [ ]:
from scipy import stats

def check_normality_homostadiscity(x, y):
  res = stats.normaltest(x)
  print("Normality first array : ", res)
  res = stats.normaltest(y)
  print("Normality second array : ", res)
  res = stats.levene(x,y)
  print("Equal variances ", res)

def run_wilcoxon_test(x, y):
  res = stats.wilcoxon(x, y)
  print("Wilcoxon test result ", res)

x = dict_sari_manual["sari_5"]
y = dict_sari_mt5["sari_5"]
#check_normality_homostadiscity(x, y)
run_wilcoxon_test(x, y)

Wilcoxon test result  WilcoxonResult(statistic=0.0, pvalue=3.809731963383853e-17)
